# Plotting up the statistical output

To gain an understanding of the meaning of the statistics, and the spatial distribution of these statistics, we  plot up several of the metrics on maps. This allows you to explore how these variables vary over the simulation domain, and interpret the meaning of these variables. 

This only gives one perspective on the choice of variables used in TCRM to simulate TC track behaviour. There are a number of other graphical products that further help with understanding the way TCRM works.

In [1]:
%matplotlib inline
from Utilities.nctools import ncLoadFile, ncGetDims, ncGetVar, ncFileInfo
from Utilities.grid import SampleGrid
from PlotInterface.maps import ArrayMapFigure, saveFigure, levels
import numpy as np
from IPython.display import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FC

from os.path import join as pjoin
import seaborn as sns
sns.set_context('paper')

In [2]:
def show(fig):
    canvas = FC(fig)
    fig

In [3]:
landmask_file = "C:/WorkSpace/tcrm/input/landmask.nc"
landmask = SampleGrid(landmask_file)


In [4]:
def getData(ncobj, varname, ij):
    var = ncGetVar(ncobj, varname)[:]
    lvar = ncGetVar(ncobj, 'l'+varname)[:]
    data = var
    data[ij] = lvar[ij]
    return data

In [5]:
path = "C:/Workspace/temp"
fname = pjoin(path, "pressure_rate_stats.nc")
ncFileInfo(fname)

In [6]:
ncobj = ncLoadFile(fname)

#lon = np.arange(90., 180., 1.0)
#lat = np.arange(-30., -5., 1.0)

lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ncobj.close()
xgrid, ygrid = np.meshgrid(lon, lat)

ls = np.zeros(np.shape(xgrid))

for i in range(len(lon)):
    for j in range(len(lat)):
        if landmask.sampleGrid(lon[i], lat[j]) > 0.0:
            ls[j, i] = 1
            
ij = np.where(ls==1)


Set the map keyword arguments that will help draw the basemap. 

In [7]:
map_kwargs = dict(llcrnrlon=100., llcrnrlat=-30,
                  urcrnrlon=160., urcrnrlat=-5.,
                  resolution='h', projection='merc')

In [9]:
ncobj = ncLoadFile(pjoin(path, "pressure_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)
ncobj.close()
fig = ArrayMapFigure()
fig.add(mudata, xgrid, ygrid, 'Mean pressure ', [950, 1000], 'Pressure (hPa)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum pressure', [900, 1000], 'Pressure (hPa)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Pressure standard deviation', [0, 50], 'Std dev.', map_kwargs)
fig.add(ardata, xgrid, ygrid, 'Pressure AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.plot()
saveFigure(fig, "pressure_stats.png")
Image("pressure_stats.png")

The first figure shows the mean pressure (in hPa) in each cell across the simulation domain. It's important to recognise that in areas with few observations, the mean may not be well-defined. TCRM makes an effort to address this, by expanding the sampling region if there are insufficient observations in an individual cell. 

Next is the minimum pressure. This is where we can see some significant variability, because the minimum pressure in any given cell depends on the vagaries of historical TC behaviour, and we are looking at an extreme value in the distribution of pressure values in each cell. 

The third figure is the standard deviation of the pressure observations. Areas where there's a lot of variation are likely near coastlines and at the outer limits of the simulation region, where TC intensity is more variable.

The final figure shows the lag-1 autocorrelation of pressure observations. For this variable, the values are all very close to 1, because the pressure values do not change dramatically from one observation to the next. For example, the largest change in pressure over a 6-hour period is only about *XX* hPa - compare that to the pressure observation and the change is (in a numerical sense) quite small. But this is not the case for all the model variables, as we will see below.

Now lets examine the pressure rate statistics. This is measuring the intensification rate of TCs in each cell, rather than the intensity (above). 

The first figure is the mean pressure rate of change. Positive values indicate an increase in pressure, negative values a decrease in pressure. This shows us the (historically) favoured regions for intensification and weakening of TCs. 

In figure 4 is the lag-1 autocorrelation. Compare the values plotted here with the corresponding figure for the pressure variable (above). We saw previously that the pressure rate of change is largely uncorrelated when looking at the autocorrelation. But what spatial patterns emerge in the pressure rate statistics? 

In [8]:
ncobj = ncLoadFile(pjoin(path, "pressure_rate_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)
ncobj.close()

fig = ArrayMapFigure()
fig.add(mudata, xgrid, ygrid, 'Mean pressure rate', [-1, 1], 'Pressure rate (hPa/hr)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum pressure rate', [-10, 10], 'Pressure rate (hPa/hr)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Pressure rate standard deviation', [0, 5], 'Std dev.', map_kwargs)
fig.add(ardata, xgrid, ygrid, 'Pressure rate AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.plot()

saveFigure(fig, "pressure_rate_stats.png")
Image("pressure_rate_stats.png")

In [10]:
ncobj = ncLoadFile(pjoin(path, "speed_rate_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)
ncobj.close()
fig = ArrayMapFigure()
fig.add(ardata, xgrid, ygrid, 'Speed rate AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.add(mudata, xgrid, ygrid, 'Mean speed rate', [-1, 1], 'Speed rate (m/s/hr)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum speed rate', [-10, 10], 'Speed rate (m/s/hr)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Speed standard deviation', [0, 5], 'Std dev.', map_kwargs)
fig.plot()
#canvas = FC(fig)
#fig
saveFigure(fig, "speed_rate_stats.png")
Image("speed_rate_stats.png")

In [11]:
ncobj = ncLoadFile(pjoin(path, "speed_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)
ncobj.close()
fig = ArrayMapFigure()
fig.add(ardata, xgrid, ygrid, 'Speed AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.add(mudata, xgrid, ygrid, 'Mean speed', [0, 25], 'Speed (m/s)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum speed', [0, 25], 'Speed (m/s)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Speed standard deviation', [0, 20], 'Std dev.', map_kwargs)
fig.plot()
#canvas = FC(fig)
#fig
saveFigure(fig, "speed_stats.png")
Image("speed_stats.png")

In [28]:
ncobj = ncLoadFile(pjoin(path, "bearing_rate_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)
ncobj.close()

In [50]:
print np.percentile(sigdata, 90)
print np.median(sigdata)

In [51]:
fig = ArrayMapFigure()
fig.add(ardata, xgrid, ygrid, 'Bearing rate AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.add(mudata, xgrid, ygrid, 'Mean bearing rate', [0, 5], 'Bearing rate (degrees/hr)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum bearing rate', [-1., 1], 'Bearing rate (degrees/hr)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Bearing rate standard deviation', [0, 15], 'Std dev.', map_kwargs)
fig.plot()
#canvas = FC(fig)
#fig
saveFigure(fig, "bearing_rate_stats.png")
Image("bearing_rate_stats.png")

In [15]:
ncobj = ncLoadFile(pjoin(path, "bearing_stats.nc"))
lon = ncGetDims(ncobj, 'lon')
lat = ncGetDims(ncobj, 'lat')
ardata = getData(ncobj, 'alpha', ij)
mudata = getData(ncobj, 'mu', ij)
mindata = getData(ncobj, 'min', ij)
sigdata = getData(ncobj, 'sig', ij)

ncobj.close()
fig = ArrayMapFigure()
fig.add(ardata, xgrid, ygrid, 'Bearing AR(1)', [-1, 1], 'AR(1)', map_kwargs)
fig.add(mudata*180./np.pi, xgrid, ygrid, 'Mean bearing', [0, 360.], 'Bearing (degrees)', map_kwargs)
fig.add(mindata, xgrid, ygrid, 'Minimum bearing', [0, 180], 'Bearing (degrees)', map_kwargs)
fig.add(sigdata, xgrid, ygrid, 'Bearing standard deviation', [0, 1], 'Std dev.', map_kwargs)
fig.plot()
#canvas = FC(fig)
#fig
saveFigure(fig, "bearing_stats.png")
Image("bearing_stats.png")